In [8]:
!pip3 install tensorflow-addons
import numpy as np
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
#from tensorflow.keras.utils import plot_model

import matplotlib.pyplot as plt
import cv2
import librosa
import time
from os import path
import os
import imageio
from scipy.io.wavfile import write
from zipfile import ZipFile
from tqdm.auto import tqdm
from glob import glob


In [7]:
from PIL import Image
import glob
import h5py
from glob import glob
import os

def makedir(path,file):
    url = os.path.join(path,file)
    if os.path.exists(url):
        pass
    else:
        os.mkdir(url)
    
    return url


filename=  'emotion_speech/true_emotion_speech/*/*.wav'
filepath = glob(filename)
count  =len(filepath)
for i, path in enumerate(filepath):
    y, sr = librosa.load(path,sr=16000)
    D = np.abs(librosa.stft(y))
    amp_max = np.amax(D)
    x1 = librosa.amplitude_to_db(D, ref=np.max)
    max_x1 = np.amax(x1)
    min_x1 = np.amin(x1)
    root = 'cycleGAN'
    path2 = path.split('\\')[-2]
    root_path=makedir(root,path2)
    true_path = root_path+"\\"+path.split('\\')[-1][:-4]+'.jpg'
    diff = max_x1 - min_x1
    imageio.imwrite(true_path, x1)




    

In [6]:
def read_audio_from_filename(filename):
    audio, sr = librosa.load(filename,sr=16000)
    return audio,sr

def write_audio_to_file(audio,name):
    audio1 = write(name,16000,audio)
    return audio1

def make_spectrogram(audio,sr):
    spec=librosa.stft(audio)
    
    liborsa.display.specshow(librosa.power_to_db(spec,ref=np.max))
    a = np.abs(spec)**2
    a = a/np.amax(a)
    a = 10*log10(a)
    a = a*255/np.amax(a)
    plt.imshow(a)
    plt.show()
    return a

def make_audio(spec):
    res = librosa.griffinlim(spec)
    return res

def convert_data(w_f,w_w_f):
    audio,sr = read_audio_from_filename(w_f)
    spec = make_spectrogram(audio,sr)
    res = make_audio(spec)
    result = write_audio_to_file(res,w_w_f)
    return result
    
    
specto = convert_data()
im = cv2.imread("spec.png",0)

TypeError: convert_data() missing 2 required positional arguments: 'w_f' and 'w_w_f'

Data base

In [5]:
class_map = {'angry':0,'disgust':1, 'fearful':2,'happy':3,'neautral':4,'sad':5,'surprised':6}
IMG_SIZE=40
def database(label):
    
    filename = 'cycleGAN\\'+label+'\\*.jpg'
    img_paths = glob(filename)
    data_count = len(img_paths)
    X = np.zeros((data_count, IMG_SIZE, IMG_SIZE))
    label = np.zeros((data_count, ))
    for i, path in tqdm(enumerate(img_paths)):
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_resize = cv2.resize(image,(IMG_SIZE,IMG_SIZE))
        img_resize = (img_resize - 127.5) / 127.5
        X[i] = img_resize
        cls = path.split('\\')[-2]
        
        label[i] = class_map[cls]
    return X,label

X_angry,label_angry = database("angry")
X_disgust,label_disgust = database('disgust')
X_fearful,label_fearful = database('fearful')
X_happy,label_happy = database('happy')
X_neautral,label_neautral = database('neautral')
X_sad, label_sad = database('sad')
X_surprised,label_surprised = database('surprised')
emo1='angry'
emo2='disgust'
filename1='thesis/'+emo1+'2'+emo2
filename2='thesis/'+emo2+'2'+emo1
print(filename1)
# save npz file
np.savez_compressed(filename1,X_angry,X_disgust)
np.savez_compressed(filename2,X_disgust,X_angry)


thesis/angry2disgust


In [8]:
def mkdir(base, name):
  path = os.path.join(base, name)
  if not os.path.exists(path):
    os.makedirs(path)
  return path

# change the following lines
main_dir = "home"
domain_a = "angry"
domain_b = "neautral"
name_of_npz = domain_a+"2"+domain_b+".npz"
path_to_npz = "/home/"+name_of_npz

fft_len = 512
spec_dim = 40
hop_length = 256

path_to_logs = mkdir(main_dir,"logs")
path_to_log = mkdir(path_to_logs,domain_a+'-'+domain_b+'_'+str(int(time.time())))
assert path_to_npz.split("/")[-1][:-4] == domain_a+"2"+domain_b

In [3]:
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_image = Input(shape=image_shape)
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)

	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)

	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)

	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)

	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	# define model
	model = Model(in_image, patch_out)
	# compile model
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
	plot_model(model, to_file='discriminator_model.png', show_shapes=True, show_layer_names=True)
	return model

# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)

	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)

	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)

	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)

	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)

	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)

	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)

	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(1, (7,7), padding='same', kernel_initializer=init)(g)

	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	plot_model(model, to_file='generator_model.png', show_shapes=True, show_layer_names=True)
	return model

# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
	# ensure the model we're updating is trainable
	g_model_1.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape)
	gen1_out = g_model_1(input_gen)
	output_d = d_model(gen1_out)
	# identity element
	input_id = Input(shape=image_shape)
	output_id = g_model_1(input_id)
	# forward cycle
	output_f = g_model_2(gen1_out)
	# backward cycle
	gen2_out = g_model_2(input_id)
	output_b = g_model_1(gen2_out)
	# define model graph
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	# define optimization algorithm configuration
	opt = Adam(lr=0.0002, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
	plot_model(model, to_file='composite_model.png', show_shapes=True, show_layer_names=True)
	return model

# load and prepare training images
def load_real_samples(filename):
	# load the dataset
	data = np.load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]

dataset = reshape((image_shape[0], image_shape[1], 1))
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# choose random instances
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = np.ones((n_samples, patch_shape, patch_shape, 1))
	return X, y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = np.zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

# save the generator models to file
def save_models(step, g_model_AtoB, g_model_BtoA):
	# save the first generator model
	filename1 = 'g_model_AtoB_%06d.h5' % (step+1)
	g_model_AtoB.save(filename1)
	# save the second generator model
	filename2 = 'g_model_BtoA_%06d.h5' % (step+1)
	g_model_BtoA.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
	# select a sample of input images
	X_in, _ = generate_real_samples(trainX, n_samples, 0)
	# generate translated images
	X_out, _ = generate_fake_samples(g_model, X_in, 0)
	# scale all pixels from [-1,1] to [0,1]
	X_in = (X_in + 1) / 2.0
	X_out = (X_out + 1) / 2.0
	# plot real images
	for i in range(n_samples):
		plt.subplot(2, n_samples, 1 + i)
		plt.axis('off')
		plt.imshow(X_in[i].reshape(167,1025), cmap='gray')
	# plot translated image
	for i in range(n_samples):
		plt.subplot(2, n_samples, 1 + n_samples + i)
		plt.axis('off')
		plt.imshow(X_out[i].reshape(167,1025), cmap='gray')
	# save plot to file
	filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
	plt.savefig(filename1)
	plt.close()

# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif np.random.random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = np.random.randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return np.asarray(selected)

# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
	# define properties of the training run
	n_epochs, n_batch, = 50, 1
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance every so often
		if (i+1) % (bat_per_epo * 1) == 0:
			# plot A->B translation
			summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
			# plot B->A translation
			summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
		if (i+1) % (bat_per_epo * 5) == 0:
			# save the models
			save_models(i, g_model_AtoB, g_model_BtoA)

# load image data
dataset = load_real_samples('thesis/angry2disgust.npz')
#dataset[0] = dataset[0][:int(0.25*len(dataset[0]))]
#dataset[1] = dataset[1][:int(0.20*len(dataset[1]))]
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

NameError: name 'dataset' is not defined

load image data

In [11]:
spec_dim=40
# load image data

def load_real_samples(filename):
	# load the dataset
	data = np.load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	return [X1, X2]
dataset = load_real_samples('thesis/angry2disgust.npz')
#dataset[0] = dataset[0][:int(0.25*len(dataset[0]))]
#dataset[1] = dataset[1][:int(0.20*len(dataset[1]))]
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]

image_shape = (spec_dim,spec_dim,1)


Loaded (301, 40, 40) (154, 40, 40)


In [12]:
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

NameError: name 'plot_model' is not defined